# Table of Contents
 <p><div class="lev1 toc-item"><a href="#NETCONF/YANG" data-toc-modified-id="NETCONF/YANG-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>NETCONF/YANG</a></div><div class="lev2 toc-item"><a href="#Useful-Snippets" data-toc-modified-id="Useful-Snippets-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Useful Snippets</a></div><div class="lev3 toc-item"><a href="#Enable-Debugging" data-toc-modified-id="Enable-Debugging-111"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Enable Debugging</a></div><div class="lev3 toc-item"><a href="#Pretty-Printing-XML" data-toc-modified-id="Pretty-Printing-XML-112"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Pretty Printing XML</a></div><div class="lev1 toc-item"><a href="#Connecting-to-a-Device" data-toc-modified-id="Connecting-to-a-Device-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Connecting to a Device</a></div><div class="lev2 toc-item"><a href="#Capabilities" data-toc-modified-id="Capabilities-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Capabilities</a></div><div class="lev2 toc-item"><a href="#YANG-Schema-Discovery" data-toc-modified-id="YANG-Schema-Discovery-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>YANG Schema Discovery</a></div><div class="lev1 toc-item"><a href="#Device-Configuration" data-toc-modified-id="Device-Configuration-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Device Configuration</a></div><div class="lev2 toc-item"><a href="#Getting-the-Configuration" data-toc-modified-id="Getting-the-Configuration-31"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Getting the Configuration</a></div><div class="lev2 toc-item"><a href="#Editing-Config" data-toc-modified-id="Editing-Config-32"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Editing Config</a></div><div class="lev1 toc-item"><a href="#Operational-Data" data-toc-modified-id="Operational-Data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Operational Data</a></div><div class="lev2 toc-item"><a href="#Verbose-Interface-Stats" data-toc-modified-id="Verbose-Interface-Stats-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Verbose Interface Stats</a></div><div class="lev2 toc-item"><a href="#Zeroing-In-On-A-Specific-Interface" data-toc-modified-id="Zeroing-In-On-A-Specific-Interface-42"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Zeroing In On A Specific Interface</a></div><div class="lev2 toc-item"><a href="#Work-with-the-Routing-Table-(RIB)" data-toc-modified-id="Work-with-the-Routing-Table-(RIB)-43"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Work with the Routing Table (RIB)</a></div><div class="lev1 toc-item"><a href="#Conclusion" data-toc-modified-id="Conclusion-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Conclusion</a></div>

# NETCONF/YANG
This notebook goes through a set of examples with a CSR1000v virtual platform. The goal is to show how NETCONF/YANG can be simply leveraged with IOS-XE to perform a range of tasks. We will cover topics like:

* Basic connectivity
* Why we really want to use some form of client library
* Getting started with a Python client like [ncclient](https://github/com/ncclient/ncclient)
* Model and feature discovery
* Querying and configuring features


## Useful Snippets

Let's define some useful little snippets of code that we might want to run to define utilities or to enable debugs, set variables for use, etc.

### Enable Debugging

The ncclient libary can generate vast amounts of debug information via standard Python logging. The code fragment below shows how to enable this. But the logging can be **vast** if set to `DEBUG`! You have been warned!

In [ ]:
import logging

handler = logging.StreamHandler()
for l in ['ncclient.transport.ssh', 'ncclient.transport.session', 'ncclient.operations.rpc']:
    logger = logging.getLogger(l)
    if not logger.hasHandlers():
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)

### Pretty Printing XML

The ncclient library works with XML. It takes XML in and gives XML back. Yeah, I know it's not JSON, so we need to make it a little easier to read, so let's define a pretty printer we can use with ncclient responses:

In [ ]:
from lxml import etree as ET

def pretty_print(element):
    """print the XML data, print a line between top level children
    so that the elements can be visually identified
    input:
        element etree element
    """
    # NC XML data embeds actual data in a '<data></data>' tag
    # which is the first and only child.
    # data = element.data.getchildren()
    data = list(element.data)
    items = len(data)
    for i, d in enumerate(data):
        if i > 0 and i < items:
            print('*' * 40)
            print()
        print(ET.tostring(d, pretty_print=True).decode('utf-8'))


# Connecting to a Device

First, we need to make sure the device itself has NETCONF/YANG enabled. It's pretty simple. Basically, SSH to the device on the NETCONF agent's port and provide the proper subsystem (netconf).

```
$ ssh -p 830 -s vagrant@172.20.20.10 netconf
vagrant@172.20.20.20's password:
<?xml version="1.0" encoding="UTF-8"?>
<hello xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
<capabilities>
<capability>urn:ietf:params:netconf:base:1.0</capability>
<capability>urn:ietf:params:netconf:base:1.1</capability>
[...]
^C
$
```

<p class='my-notify-info'>The -s is a switch to SSH to interpret the provided remote command as the SSH subsystem to use.</p>

Next, let's define some variables that let us attach to the device:

In [ ]:
HOST = '172.20.20.10'
PORT = 830
USER = 'vagrant'
PASS = 'vagrant'

Now let's establish a NETCONF session to that box using ncclient (this next step takes a second or two, please watch out for the `In [*]`, indicating that the Python kernel is still busy):

In [ ]:
from ncclient import manager

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb, device_params={'name':'csr'})

## Capabilities
If you observe the output from the `connect` command carefully then you'll see that the `server_capabilities` have been returned. Looks like a dictionary!. Let's look at those capabilities:

In [ ]:
caps = m.server_capabilities
[c for c in caps]

Ok, that's a lot to look at, so show only NETCONF capabilities:

In [ ]:
[c for c in caps if c.startswith('urn:ietf:params:netconf')]

And now let's look at the capabilities that are related to model support:

In [ ]:
import re

for c in caps:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

## YANG Schema Discovery

Building on the parsing of the capabilities we saw above, NETCONF/YANG can also let a client discover more details on the schemas supported by a box.

But why do we care? Let's think back to what we talked about earlier. About the need for boxes to describe themselves to their clients. To expose their "model".

Let's pick a base model that looks like it may do something interesting, for example ```Cisco-IOS-XR-ifmgr-cfg```, and let's download the schema. The ncclient library provides a nice, simple function for that. Again, the next operation takes a little while to complete).

In [ ]:
c = m.get_schema('ietf-interfaces')
# c = m.get_schema('iana-if-type')
# c = m.get_schema('ned')
print(c)

<p class="my-notify-info">Try with different models. Look at the capabilities output from above. Look for other models of interest and their names and try to retrieve them via `get_schema`.</p>

The YANG model is actually embedded in XML which is a bit hard to look at so let's only print the data part of the schema which represents the vanilla YANG model:

In [ ]:
print(c.data)

If you look at the above output you'll notice that it looks a lot like C-code with all the curly braces, type definitions etc. In fact, a programmer might already *grasp* the purpose of a YANG model definition by looking at the vanilla YANG text.

As a 'normal' user we might want to transform that YANG text into something more readable. We use the `pyang` tool for that purpose. With the this tool (which is also written in Python) we can transform the YANG model (stored in `c.data` as a string) as a tree representation:

In [ ]:
import subprocess
print(subprocess.run(["pyang", "-f", "tree"], input=c.data, stdout=subprocess.PIPE, universal_newlines=True).stdout)

pyang can convert YANG models into different output. The below command shows us the different formats it knows about.

In [ ]:
!pyang -h | grep -e '--format' -A3

And now let's write the output in a different format (a JavaScript enabled HTML format) into a file so that we can reference it later:

In [ ]:
with open('/home/docker/notebooks/tmp/ietf-interfaces.html', 'w') as fh:
    subprocess.run(["pyang", "-f", "jstree"], input=c.data, stdout=fh, universal_newlines=True)

In [ ]:
%%HTML
<iframe width="100%" height="400px" src='/notebooks/tmp/ietf-interfaces.html'</iframe>

You should now be able to open this file by clicking [here](/notebooks/tmp/ietf-interfaces.html). Keep the window open as we're going to reference it later in the RESTCONF section as well.

# Device Configuration

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* `get_config` - takes a target data store and an optional filter
* `edit_config` - takes a target data store and an XML document with the edit request


## Getting the Configuration

Let's look at some simple requests... The next two statements retrieve the **entire** configuration and simply *pretty-print* it for better readability.

In [ ]:
c = m.get_config(source='running')
pretty_print(c)

Now, since we have the data in an XML object we can also programatically deal with it. How about getting al the interfaces from the XML tree and then print some of their attributes? We use a library function `findall()` and instruct it to look for the relevant elements. Refer to [lxml documentation](http://lxml.de/) for more detailed information about the lxml library.

<p class="my-notify-info">The namespace is required to distinguish between elements with the same name in different namespaces. In this case, we do have the `ietf-interfaces` namespace and the `native` namespace which contain both interface elements. At the end of the day they both point to the same router interfaces but we have to be specific with our find!</p>

In [ ]:
# ietf-interfaces name space
namespace = "urn:ietf:params:xml:ns:yang:ietf-interfaces"

interfaces = c.data.findall('.//{%s}interface' % namespace)
for ifc in interfaces:
    print(ifc.find('{%s}name' % namespace).text, ": ", sep='', end='')
    print(ifc.find('{%s}enabled' % namespace).text)

We can use a filter to only return parts of the XML model tree. In this case, we're only interested in the `ip domain-name` configuration part. We define the filter accordingly:

In [ ]:
filter = '''
<native>
  <ip><domain/></ip>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Or get the hostname of the device from the native mode:

In [ ]:
filter = '''
<native>
  <hostname/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

As we should see by the returned data, the **`name`** element data has been filled with the actual domain name and hostname (respectively) that is configured on the device!

Now let's add another simple filter to just get interface configuration:

In [ ]:
filter = '''
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Let's refine the query a bit to look at **just** the 2nd Ethernet port:

In [ ]:
filter = '''
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <name>GigabitEthernet2</name>
  </interface>
</interfaces>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

Or we may want to look at just routing configuration. To make this a bit more interesting, we've added an OSPF router on both devices with a few static routes that get distributed into OSPF.

<p class="my-notify-info">The below XML output is CONFIGURATION data. Note that the actual routing table (RIB) is OPERATIONAL data. We'll look at that in a minute!</p>

In [ ]:
filter = '<routing xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">'
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

## Editing Config

Let's start with something simple, like applying an IP address to an interface and bringing it up. We can actually use the data we got from the existing `loopback99` interface as a template. As you might have noticed above, the interface is configured but currently in *administrative shutdown* state.

To do this, we use two functions:

* ```edit_config``` on the candidate data store
* ```commit``` to commit the change to running config, just like on the CLI

For good measure, we'll also get the interface config back again to check out that what we asked to happen, actually did happen. By using the `etree.XML()` function we automatically check correct syntax as well.

In [ ]:
loopback = ET.XML('''
<config>
 <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <name>Loopback99</name>
    <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:softwareLoopback</type>
    <enabled>false</enabled>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
      <address>
        <ip>1.2.3.4</ip>
        <netmask>255.0.0.0</netmask>
      </address>
    </ipv4>
    <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
      <address>
        <ip>2001:db8:99::1</ip>
        <prefix-length>64</prefix-length>
      </address>
    </ipv6>
  </interface>
 </interfaces>
</config>
''')

filter = '''
<interfaces>
  <interface>
    <name>Loopback99</name>
  </interface>
</interfaces>
'''

In [ ]:
from ncclient.operations.rpc import RPCError

print(ET.tostring(loopback, pretty_print=True).decode('utf-8'))
print("#" * 40, end='\n\n')

try:
    m.edit_config(loopback, target='running', format='xml')
except RPCError as e:
    print(e.tag, e.severity)
    xml = ET.XML(e.info.encode('utf-8'))
    print(ET.tostring(xml, pretty_print=True).decode('utf-8'))

# Candidate configuration store is not supported on IOS-XE as of today
# This works on IOS XR, though.
# m.commit()

newconfig = m.get_config(source='running', filter=('subtree', filter))
pretty_print(newconfig)

You can go ahead and change IP addresses or remove one of the address families and re-run the above code snippet. Note that these configuration changes go into the 'running' configuration store. So they are not automatically persistent (e.g. they do not survive a reboot).

# Operational Data

As we touched on before, NETCONF also has the ```get``` operation. This can get both configuration state **and** operational state.

## Verbose Interface Stats

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"/>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Zeroing In On A Specific Interface

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
    </interface>
  </interface>
</interfaces-state>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Work with the Routing Table (RIB)
The routing table is also operational data. We can use the `ietf-routing` YANG model to retrieve the current routing table and work with that data. In this case we're specifying a filter that only retrieves the IPv4 unicast-routing table. But we could also filter for IPv6 and/or multicast routing tables, if they exist.

In [ ]:
from ncclient.operations import TimeoutExpiredError

filter = '''
<routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
  <routing-instance>
    <ribs>
      <rib>
        <name>ipv4-default</name>
        <address-family xmlns:v4ur="urn:ietf:params:xml:ns:yang:ietf-ipv4-unicast-routing">v4ur:ipv4-unicast</address-family>
        <default-rib>true</default-rib>
        <routes/>
      </rib>
    </ribs>
  </routing-instance>
</routing-state>
'''

try:
    c = m.get(filter=('subtree', filter))
    pretty_print(c)
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))


That's quite a mouthful of XML output. We are going to use *XML ElementTree* to digest this information and extract the pieces we're actually interested in.

For this, we need to have the correct namespace (defined in `ns`) and a namespace map. The prefix doesn't actually matter in this case since all the returned items are assuming a default namespace, e.g. the prefix is empty.

But for the `xpath` method to work we need to have a namespace map and no prefix in the namespace map is throwing an error. Therefore we define a namespace map with a 'bogus' prefix, 'rt'. It could be anything in this particular case to make it work.

So, the code does:

- find all `route` elements in the `ietf-routing` namespace
- for all of these routes, using the `xpath` method, it extracts the text from these sub-elements
    - destination-prefix
    - next-hop-address
    - outgoing-interface
  (note that one of the last two can be empty)
- print them with an index

In [ ]:
ns = 'urn:ietf:params:xml:ns:yang:ietf-routing'
nsmap = dict(rt=ns)

routes = c.data.findall(".//{%s}route" % ns)
for i, route in enumerate(routes):
    dest = route.xpath('./rt:destination-prefix/text()', namespaces=nsmap)
    hop = route.xpath('./rt:next-hop/rt:next-hop-address/text()', namespaces=nsmap)
    intfc = route.xpath('./rt:next-hop/rt:outgoing-interface/text()', namespaces=nsmap)
    print("%4d:\t%-20s via " % (i, dest[0]), end='')
    if len(hop) > 0:
        print("%s " % hop[0], end='')
    if len(intfc) > 0:
        print("%s" % intfc[0], end='')
    print()
    

# Conclusion

NETCONF presents a number of useful primitives that we looked at today:

* **get config** to retrieve *configuration* data
* **get** to retrieve *operational* data
* **edit config** allows us to edit / change the configuration of the device
* **commit** commits the changes from the candidate store into the running store
* **get schema** retrieves a YANG schema from the device

There are more, so follow up to understand more later.

With the primitives we ran through today, you can do basic model discovery, understand what features are supported, and understand what parts of models are not supported.

Products such as NSO, or open source projects like ODL or the YDK can use these basic capabilities to work with devices in a much more reliable way.